In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Collaborative Filtering**

In [ ]:
#Collaborative Filtering:
#importing libraries
import pandas as pd
import numpy as np

In [ ]:
movies=pd.read_csv('/content/drive/My Drive/movie.csv')
rating=pd.read_csv('/content/drive/My Drive/rating.csv')

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [ ]:
rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [ ]:
#reduce size of data to prevent running out of memory
x=rating['userId'].value_counts()>300

In [ ]:
y = x[x].index

In [ ]:
rating=rating[rating['userId'].isin(y)]
rating

,userId,movieId,rating,timestamp
960,11,1,4.5,2009-01-02 01:13:41
961,11,10,2.5,2009-01-02 01:15:59
962,11,19,3.5,2009-01-01 04:21:44
963,11,32,5.0,2009-01-01 04:11:35
964,11,39,4.5,2009-01-02 01:17:12
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [ ]:
df = pd.merge(rating,movies,on='movieId')

In [ ]:
df.head

<bound method NDFrame.head of          userId  ...                                       genres
0            11  ...  Adventure|Animation|Children|Comedy|Fantasy
1            24  ...  Adventure|Animation|Children|Comedy|Fantasy
2            54  ...  Adventure|Animation|Children|Comedy|Fantasy
3            58  ...  Adventure|Animation|Children|Comedy|Fantasy
4            91  ...  Adventure|Animation|Children|Comedy|Fantasy
...         ...  ...                                          ...
9891287  138301  ...                                 Comedy|Crime
9891288  138301  ...                       Action|Comedy|Thriller
9891289  138301  ...                                    Drama|War
9891290  138406  ...                                  Crime|Drama
9891291  138406  ...                                    Film-Noir

[9891292 rows x 6 columns]>

In [ ]:
#dropping unnecessary feat
df.drop(columns = ['genres', 'timestamp'], inplace = True)

In [ ]:
df

,userId,movieId,rating,title
0,11,1,4.5,Toy Story (1995)
1,24,1,4.0,Toy Story (1995)
2,54,1,4.0,Toy Story (1995)
3,58,1,5.0,Toy Story (1995)
4,91,1,4.0,Toy Story (1995)
...,...,...,...,...
9891287,138301,121017,3.5,The Gentleman from Epsom (1962)
9891288,138301,121019,4.5,The Great Spy Chase (1964)
9891289,138301,121021,4.5,Taxi for Tobruk (1961)
9891290,138406,110167,4.5,"Judge and the Assassin, The (Juge et l'assassi..."


In [ ]:
df = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

In [ ]:
movie_ratingCount

,title,totalRatingCount
0,#chicagoGirl: The Social Network Takes on a Di...,2
1,$ (Dollars) (1971),9
2,$5 a Day (2008),13
3,$9.99 (2008),25
4,$ellebrity (Sellebrity) (2012),1
...,...,...
26347,À nous la liberté (Freedom for Us) (1931),139
26348,À propos de Nice (1930),4
26349,Árido Movie (2005),1
26350,Åsa-Nisse - Wälkom to Knohult (2011),2


In [ ]:
rating_with_totalRatingCount = df.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

In [ ]:
rating_with_totalRatingCount

,userId,movieId,rating,title,totalRatingCount
0,11,1,4.5,Toy Story (1995),12625
1,24,1,4.0,Toy Story (1995),12625
2,54,1,4.0,Toy Story (1995),12625
3,58,1,5.0,Toy Story (1995),12625
4,91,1,4.0,Toy Story (1995),12625
...,...,...,...,...,...
9891287,138301,121017,3.5,The Gentleman from Epsom (1962),1
9891288,138301,121019,4.5,The Great Spy Chase (1964),1
9891289,138301,121021,4.5,Taxi for Tobruk (1961),1
9891290,138406,110167,4.5,"Judge and the Assassin, The (Juge et l'assassi...",1


In [ ]:
#to reduce sparcity 
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')

In [ ]:
rating_popular_movie

,userId,movieId,rating,title,totalRatingCount
0,11,1,4.5,Toy Story (1995),12625
1,24,1,4.0,Toy Story (1995),12625
2,54,1,4.0,Toy Story (1995),12625
3,58,1,5.0,Toy Story (1995),12625
4,91,1,4.0,Toy Story (1995),12625
...,...,...,...,...,...
9866656,130477,67068,3.0,"Quick and the Dead, The (1987)",53
9866657,137920,67068,4.0,"Quick and the Dead, The (1987)",53
9884334,61050,128862,2.5,Casanova (2005),419
9886462,74136,80330,2.0,Offside (2006),70


In [ ]:
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)

In [ ]:
movie_features_df

userId,11,24,54,58,91,96,104,116,131,132,133,134,137,147,156,208,215,220,245,247,251,258,271,278,283,294,295,298,309,311,318,337,347,348,359,367,370,375,387,388,...,138166,138170,138177,138186,138191,138196,138200,138201,138202,138205,138208,138211,138254,138259,138267,138270,138285,138289,138295,138301,138307,138317,138325,138335,138343,138382,138386,138387,138397,138404,138406,138411,138414,138436,138437,138454,138456,138472,138474,138493
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0
'night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,4.5,0.0,3.5,0.0,0.0,4.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.5,0.0,0.0,3.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loudQUIETloud: A Film About the Pixies (2006),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,2.5,0.0,0.0,4.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#fitting model and calculating csr matrix
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
distances, indices = model_knn.kneighbors(movie_features_df.iloc[7922,:].values.reshape(1, -1), n_neighbors = 11)

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[7922]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Shutter Island (2010):

1: Inception (2010), with distance of 0.2598086747230971:
2: Inglourious Basterds (2009), with distance of 0.31711588703580607:
3: District 9 (2009), with distance of 0.3442118225630064:
4: Hangover, The (2009), with distance of 0.3526340200818807:
5: Avatar (2009), with distance of 0.3532000524270281:
6: Black Swan (2010), with distance of 0.3563514047779057:
7: Dark Knight, The (2008), with distance of 0.367631670974486:
8: Kick-Ass (2010), with distance of 0.3695878590095194:
9: Social Network, The (2010), with distance of 0.38157801077384756:
10: Sherlock Holmes (2009), with distance of 0.3828316453592223:


In [ ]:
distances, indices = model_knn.kneighbors(movie_features_df.iloc[80,:].values.reshape(1, -1), n_neighbors = 11)

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[80]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 3 Idiots (2009):

1: My Name is Khan (2010), with distance of 0.7673367062253351:
2: Secret in Their Eyes, The (El secreto de sus ojos) (2009), with distance of 0.783620010263091:
3: Limitless (2011), with distance of 0.8013132222012815:
4: Intouchables (2011), with distance of 0.806180742346394:
5: Departures (Okuribito) (2008), with distance of 0.809650758166444:
6: Ip Man (2008), with distance of 0.8160144617010221:
7: Inception (2010), with distance of 0.8215581608467655:
8: Perks of Being a Wallflower, The (2012), with distance of 0.8231459066419848:
9: Shutter Island (2010), with distance of 0.8232314356432175:
10: King's Speech, The (2010), with distance of 0.8253186720799703:


# Content based

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
movies = pd.read_csv("/content/drive/My Drive/movie.csv")
genome_scores = pd.read_csv("/content/drive/My Drive/genome_scores.csv")
genome_tags = pd.read_csv("/content/drive/My Drive/genome_tags.csv")
tag = pd.read_csv("/content/drive/My Drive/tag.csv")

In [ ]:
tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [ ]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [ ]:
genome_scores

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
11709763,131170,1124,0.58775
11709764,131170,1125,0.01075
11709765,131170,1126,0.01575
11709766,131170,1127,0.11450


In [ ]:
genome_scores = genome_scores[genome_scores['relevance']> 0.5].merge(genome_tags,on='tagId',how='left') 

In [ ]:
genome_scores

,movieId,tagId,relevance,tag
0,1,11,0.57700,3d
1,1,19,0.67050,action
2,1,29,0.89200,adventure
3,1,30,0.67625,affectionate
4,1,61,0.54475,animal movie
...,...,...,...,...
459567,131170,1092,0.59700,visually stunning
459568,131170,1100,0.52175,watch the credits
459569,131170,1104,0.70550,weird
459570,131170,1123,0.64375,writers


In [ ]:
genome_scores = genome_scores.groupby('movieId')['tag'].apply(' '.join).reset_index()

In [ ]:
genome_scores

,movieId,tag
0,1,3d action adventure affectionate animal movie ...
1,2,action adapted from:book adventure animal movi...
2,3,chase comedy crappy sequel destiny family fish...
3,4,adultery betrayal chick flick divorce feel goo...
4,5,catastrophe chase comedy crappy sequel culture...
...,...,...
10376,130578,action assassin assassination assassins death ...
10377,130840,atmospheric bad ending beautiful beautiful sce...
10378,131013,absurd buddy movie coen bros comedy crude humo...
10379,131168,atmospheric beautiful scenery beautifully film...


In [ ]:
final_dataset = pd.merge(movies,genome_scores,on='movieId',how='left')

In [ ]:
final_dataset

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3d action adventure affectionate animal movie ...
1,2,Jumanji (1995),Adventure|Children|Fantasy,action adapted from:book adventure animal movi...
2,3,Grumpier Old Men (1995),Comedy|Romance,chase comedy crappy sequel destiny family fish...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,adultery betrayal chick flick divorce feel goo...
4,5,Father of the Bride Part II (1995),Comedy,catastrophe chase comedy crappy sequel culture...
...,...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy,NaN
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,NaN
27275,131258,The Pirates (2014),Adventure,NaN
27276,131260,Rentun Ruusu (2001),(no genres listed),NaN


In [ ]:
tag = tag.rename(columns = {"tag":"keywords"})
tag['keywords'].fillna('',inplace=True)
tag = tag.groupby('movieId')['keywords'].apply(' '.join).reset_index()

In [ ]:
final_dataset = pd.merge(final_dataset,tag,on='movieId',how='left')

In [ ]:
final_dataset.head()

,movieId,title,genres,tag,keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3d action adventure affectionate animal movie ...,Watched computer animation Disney animated fea...
1,2,Jumanji (1995),Adventure|Children|Fantasy,action adapted from:book adventure animal movi...,time travel adapted from:book board game child...
2,3,Grumpier Old Men (1995),Comedy|Romance,chase comedy crappy sequel destiny family fish...,old people that is actually funny sequel fever...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,adultery betrayal chick flick divorce feel goo...,chick flick revenge characters chick flick cha...
4,5,Father of the Bride Part II (1995),Comedy,catastrophe chase comedy crappy sequel culture...,Diane Keaton family sequel Steve Martin weddin...


In [ ]:
final_dataset['keywords'] = final_dataset['keywords'] + " " +final_dataset['tag'] +  " " + \
    final_dataset['genres'].str.replace("|"," ")
final_dataset['keywords'].fillna("",inplace=True)

In [ ]:
final_dataset.drop(['tag','genres'],inplace=True,axis=1)

In [ ]:
final_dataset.head()

,movieId,title,keywords
0,1,Toy Story (1995),Watched computer animation Disney animated fea...
1,2,Jumanji (1995),time travel adapted from:book board game child...
2,3,Grumpier Old Men (1995),old people that is actually funny sequel fever...
3,4,Waiting to Exhale (1995),chick flick revenge characters chick flick cha...
4,5,Father of the Bride Part II (1995),Diane Keaton family sequel Steve Martin weddin...


In [ ]:
tfidf= TfidfVectorizer()
X = tfidf.fit_transform(final_dataset['keywords'])

In [ ]:
cosine_sim = cosine_similarity(X)

In [ ]:
def get_movie_recommendation(movie_name):
    idx = final_dataset[final_dataset['title'].str.contains(movie_name)].index
    print(idx)
    if len(idx):
        movie_indices = sorted(list(enumerate(cosine_sim[idx[0]])), key=lambda x: x[1], reverse=True)[1:11]
        movie_indices = [i[0] for i in movie_indices]
        return movie_indices
    else : 
        return []

In [ ]:
title = "Shutter Island"
recommended_movie_list = get_movie_recommendation(title)

Int64Index([14869], dtype='int64')


In [ ]:
recommended_movie_list

[11362, 9588, 22191, 1571, 5857, 24592, 13384, 11484, 13336, 15534]

In [ ]:
movies.iloc[recommended_movie_list].set_index('movieId')

,title,genres
movieId,,
48516,"Departed, The (2006)",Crime|Drama|Thriller
30812,"Aviator, The (2004)",Drama
106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
1625,"Game, The (1997)",Drama|Mystery|Thriller
5956,Gangs of New York (2002),Crime|Drama
116397,Stonehearst Asylum (2014),Thriller
65882,"Uninvited, The (2009)",Drama|Horror|Mystery|Thriller
49530,Blood Diamond (2006),Action|Adventure|Crime|Drama|Thriller|War
65567,Passengers (2008),Drama|Mystery|Thriller


In [ ]:
title = "PK"
recommended_movie_list = get_movie_recommendation(title)

Int64Index([25408], dtype='int64')


In [ ]:
movies.iloc[recommended_movie_list].set_index('movieId')

,title,genres
movieId,,
73881,3 Idiots (2009),Comedy|Drama|Romance
74787,My Name is Khan (2010),Drama|Romance
3183,"Third Miracle, The (1999)",Drama
87194,"Way, The (2010)",Drama
98956,Barfi! (2012),Comedy|Drama|Romance
107883,The Lunchbox (2013),Drama|Romance
6776,Lagaan: Once Upon a Time in India (2001),Comedy|Drama|Musical|Romance
111320,Mom's Night Out (2014),Comedy
33781,Quo Vadis (1951),Drama|Romance
